# Group Project
#### **FINN43815 - Python for Finance**

In [72]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import kurtosis, skew
import yfinance as yf

### Moving Average Crossover Strategy

#### 1. Import Data

In [73]:
data = pd.read_csv('cleaned_data.csv')

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_21676/1299969751.py:1: DtypeWarning:

Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.



In [74]:
# drop unnecessary columns
data = data[["date", "permno", "price"]]
data

,date,permno,price
0,1990-01-31,0111145D UN Equity,20.6875
1,1990-02-28,0111145D UN Equity,22.0000
2,1990-03-30,0111145D UN Equity,19.5000
3,1990-04-30,0111145D UN Equity,17.6250
4,1990-05-31,0111145D UN Equity,19.8125
...,...,...,...
514691,2022-04-29,J UN Equity,138.5500
514692,2022-05-31,J UN Equity,140.0900
514693,2022-06-30,J UN Equity,127.1300
514694,2022-07-29,J UN Equity,137.3000


#### 2. Calculate monthly returns

In [75]:
data['rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change)
data.head()

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_21676/1728035520.py:1: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.



,date,permno,price,rets
0,1990-01-31,0111145D UN Equity,20.6875,NaN
1,1990-02-28,0111145D UN Equity,22.0000,0.063444
2,1990-03-30,0111145D UN Equity,19.5000,-0.113636
3,1990-04-30,0111145D UN Equity,17.6250,-0.096154
4,1990-05-31,0111145D UN Equity,19.8125,0.124113


#### 3. Calculate moving averages

In [76]:
data["sm5"] = data.groupby("permno")["price"].rolling(window=5, min_periods=5).mean().reset_index(drop=True)
data["sm15"] = data.groupby("permno")["price"].rolling(window=15, min_periods=15).mean().reset_index(drop=True)
data

,date,permno,price,rets,sm5,sm15
0,1990-01-31,0111145D UN Equity,20.6875,NaN,NaN,NaN
1,1990-02-28,0111145D UN Equity,22.0000,0.063444,NaN,NaN
2,1990-03-30,0111145D UN Equity,19.5000,-0.113636,NaN,NaN
3,1990-04-30,0111145D UN Equity,17.6250,-0.096154,NaN,NaN
4,1990-05-31,0111145D UN Equity,19.8125,0.124113,19.925,NaN
...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,0.005370,200.662,192.782667
514692,2022-05-31,J UN Equity,140.0900,0.011115,186.042,193.828667
514693,2022-06-30,J UN Equity,127.1300,-0.092512,180.462,194.789333
514694,2022-07-29,J UN Equity,137.3000,0.079997,178.242,195.424000


In [77]:
data = data.dropna()

#### 4. Identify Trading Signals

In [78]:
data["signal"] = np.where(data["sm15"] > data["sm5"], 1, 0)
data

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_21676/1324855623.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,permno,price,rets,sm5,sm15,signal
14,1991-03-28,0111145D UN Equity,20.5625,-0.003030,21.3875,20.675000,0
15,1991-04-30,0111145D UN Equity,21.3750,0.039514,21.0375,20.720833,0
16,1991-05-31,0111145D UN Equity,21.0000,-0.017544,20.8375,20.654167,0
17,1991-06-28,0111145D UN Equity,21.6250,0.029762,21.0375,20.795833,0
18,1991-07-31,0111145D UN Equity,21.0625,-0.026012,21.1250,21.025000,0
...,...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,0.005370,200.6620,192.782667,0
514692,2022-05-31,J UN Equity,140.0900,0.011115,186.0420,193.828667,1
514693,2022-06-30,J UN Equity,127.1300,-0.092512,180.4620,194.789333,1
514694,2022-07-29,J UN Equity,137.3000,0.079997,178.2420,195.424000,1


In [79]:
data.loc[:,"position"] = data.groupby("permno")["signal"].diff()
# position = 1 -> buy
# position = -1 -> sell

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_21676/281602730.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [80]:
# shift position (trading decision) by two days
data.loc[:,"position"] = data.groupby("permno")["position"].shift(2)

In [81]:
data

,date,permno,price,rets,sm5,sm15,signal,position
14,1991-03-28,0111145D UN Equity,20.5625,-0.003030,21.3875,20.675000,0,NaN
15,1991-04-30,0111145D UN Equity,21.3750,0.039514,21.0375,20.720833,0,NaN
16,1991-05-31,0111145D UN Equity,21.0000,-0.017544,20.8375,20.654167,0,NaN
17,1991-06-28,0111145D UN Equity,21.6250,0.029762,21.0375,20.795833,0,0.0
18,1991-07-31,0111145D UN Equity,21.0625,-0.026012,21.1250,21.025000,0,0.0
...,...,...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,0.005370,200.6620,192.782667,0,0.0
514692,2022-05-31,J UN Equity,140.0900,0.011115,186.0420,193.828667,1,0.0
514693,2022-06-30,J UN Equity,127.1300,-0.092512,180.4620,194.789333,1,0.0
514694,2022-07-29,J UN Equity,137.3000,0.079997,178.2420,195.424000,1,1.0


In [82]:
# get returns for strategy
data["mavg_rets"] = data.groupby("permno")["signal"].shift(2) * data.rets

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_21676/2007258890.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### 5. Calculate Trading Return

In [83]:
strategy_returns = data.groupby("date")["mavg_rets"].mean().reset_index()

In [84]:
strategy_returns.date = pd.to_datetime(strategy_returns.date)

In [85]:
strategy_returns = strategy_returns.dropna()

In [86]:
strategy_returns["mavg_cum_rets"] = (strategy_returns.mavg_rets +1).cumprod()

### Evaluation

In [87]:
px.line(strategy_returns, y='mavg_cum_rets', x='date', labels={'mavg_cum_rets': 'Cumulative returns'}, title = 'Moving Average Returns').show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [88]:
px.histogram(strategy_returns[['mavg_rets']]).show()

In [89]:
monthly_rets = strategy_returns[["date", "mavg_rets"]].resample('M', on='date').mean() # Calculate average daily return for each month
monthly_rets['Colour']='red' # Add column with 'red' value
monthly_rets.loc[monthly_rets['mavg_rets']>=0, 'Colour']='green' # Replace 'red' with 'green' where return is non-negative
px.bar(monthly_rets,x=monthly_rets.index,y="mavg_rets").update_traces(marker_color=monthly_rets["Colour"]).show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [90]:
# mean and volatility
mean_ret = strategy_returns['mavg_rets'].mean()
volatility = strategy_returns['mavg_rets'].std()
print(mean_ret, volatility)

0.005098530513870373 0.025813938043749433


In [91]:
print('Annualised expected return is: '+str(round(100*mean_ret*12,2)) +'%')
print('Annualised volatility is: '+str(round(100*volatility*np.sqrt(12),2)) +'%')

Annualised expected return is: 6.12%
Annualised volatility is: 8.94%


In [92]:
# describing statistics of returns
(strategy_returns['mavg_rets']*100).describe()

count    376.000000
mean       0.509853
std        2.581394
min      -13.908958
25%       -0.361716
50%        0.364301
75%        1.231619
max       21.874763
Name: mavg_rets, dtype: float64

In [93]:
# skewness
print(skew(strategy_returns['mavg_rets'].dropna()))

1.171007302882262


In [94]:
# kurtosis
print(kurtosis(strategy_returns["mavg_rets"].dropna()))

16.68776608001451


In [95]:
# Value at Risk
VaR = strategy_returns['mavg_rets'].quantile(0.005)
VaR

-0.08361516256905381

In [96]:
for q in [0.1, 0.05, 0.01, 0.001, 0.0001]:
    VaR = strategy_returns['mavg_rets'].quantile(q)
    print('There is a '+str(round(100*q,3))+'% chance that my losses will be '+str(round(VaR*100, 2))+'% or worse over the next day.')

There is a 10.0% chance that my losses will be -1.36% or worse over the next day.
There is a 5.0% chance that my losses will be -2.92% or worse over the next day.
There is a 1.0% chance that my losses will be -6.77% or worse over the next day.
There is a 0.1% chance that my losses will be -12.45% or worse over the next day.
There is a 0.01% chance that my losses will be -13.76% or worse over the next day.


In [97]:
VaR = strategy_returns['mavg_rets'].quantile(0.005)
rets_tail = strategy_returns.loc[strategy_returns['mavg_rets']<=VaR,["date", "mavg_rets", "mavg_cum_rets"]]
rets_tail

,date,mavg_rets,mavg_cum_rets
211,2008-10-31,-0.139090,2.288955
215,2009-02-27,-0.100134,1.822916


In [98]:
ES = rets_tail['mavg_rets'].mean()
ES

-0.1196117599263998

In [99]:
for q in [0.1, 0.05, 0.01, 0.001, 0.0001]:
    VaR = strategy_returns['mavg_rets'].quantile(q)
    rets_tail = strategy_returns.loc[strategy_returns['mavg_rets']<=VaR,:]
    ES = rets_tail['mavg_rets'].mean()
    print('I expect to lose on average '+ str(round(ES*100, 2))+'% over 1-day period given that I have exceeded my VaR for '+str(round(100*q,3))+'%.')

I expect to lose on average -3.78% over 1-day period given that I have exceeded my VaR for 10.0%.
I expect to lose on average -5.53% over 1-day period given that I have exceeded my VaR for 5.0%.
I expect to lose on average -9.71% over 1-day period given that I have exceeded my VaR for 1.0%.
I expect to lose on average -13.91% over 1-day period given that I have exceeded my VaR for 0.1%.
I expect to lose on average -13.91% over 1-day period given that I have exceeded my VaR for 0.01%.


In [100]:
# according to bloomberg the risk free rate (treasury yields) 4.76 % (last 30 years)
# calculation of sharpe ratio

rf = 0.0476
sharpe_ratio = (mean_ret-rf)/volatility
sharpe_ratio

-1.6464543075178297

In [101]:
# get SP500 data from 1990-01-31 to 2022-08-30	
sp500_ticker = yf.Ticker("^GSPC")
sp500 = sp500_ticker.history(start="1990-02-28", end="2022-09-30", interval="1mo")
sp500 = sp500[["Close"]]
sp500["Rets"] = sp500.Close.pct_change()

In [102]:
sp500

,Close,Rets
Date,,
1990-03-01 00:00:00-05:00,339.940002,NaN
1990-04-01 00:00:00-05:00,330.799988,-0.026887
1990-05-01 00:00:00-04:00,361.230011,0.091989
1990-06-01 00:00:00-04:00,358.019989,-0.008886
1990-07-01 00:00:00-04:00,356.149994,-0.005223
...,...,...
2022-05-01 00:00:00-04:00,4132.149902,0.000053
2022-06-01 00:00:00-04:00,3785.379883,-0.083920
2022-07-01 00:00:00-04:00,4130.290039,0.091116


In [103]:
# Calculate the average return of the market
average_market_return = sp500.Rets.mean()

# Calculate the covariance between portfolio and market returns
covariance = strategy_returns['mavg_rets'].cov(sp500['Rets'].reset_index(drop=True))

# Calculate the variance of the market returns
market_variance = sp500['Rets'].var()

# Calculate the portfolio's beta
portfolio_beta = covariance / market_variance

# Calculate Jensen's Alpha
jensens_alpha = mean_ret - (rf + portfolio_beta * (average_market_return - rf))

print("Portfolio Beta:", portfolio_beta)
print("Jensen's Alpha:", jensens_alpha)

Portfolio Beta: 0.018516648320690266
Jensen's Alpha: -0.04174945589360124


In [104]:
# treynor ratio
treynor_ratio = (mean_ret - rf)/portfolio_beta
treynor_ratio

-2.2953111572918425

In [105]:
# maximum drawdown
peak = strategy_returns.reset_index(drop=True).loc[1, "mavg_cum_rets"]
max_drawdown = 0
for ret in strategy_returns["mavg_cum_rets"].dropna():
    if ret >= peak:
        peak = ret
    else:
        drawdown = (peak - ret)/peak
        max_drawdown = max(max_drawdown, drawdown)

print(max_drawdown)

0.38234448312669506


In [106]:
# calmar-ratio
annualised_exp_return = round(mean_ret*12,2)
calmar_ratio = (annualised_exp_return - rf)/max_drawdown
calmar_ratio

0.03243148664941266